In [38]:
import eumdac
import os
import requests
import shutil
import time

In [39]:
consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('SECRET')

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)


In [40]:
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
latest = selected_collection.search().first()

In [41]:
datatailor = eumdac.DataTailor(token)

In [42]:
chain = eumdac.tailor_models.Chain(
    product='HRSEVIRI',
    format='netcdf4',
    roi={'NSWE': [30,10,-10,0]}
)

In [43]:
# Send the customisation to Data Tailor Web Services
customisation = datatailor.new_customisation(latest, chain)

In [46]:
status = customisation.status
while status!='DONE': #quick'n'dirty, we should check for errors output
    status= customisation.status
    print(customisation.status)
    time.sleep(10)
print(status)

DONE


In [47]:
customisation.outputs

['HRSEVIRI-C-5da3585f/HRSEVIRI_20230914T102700Z_20230914T102700Z_epct_5da3585f_C.nc']

In [37]:
with customisation.stream_output(customisation.outputs[0]) as stream, \
            open(stream.name, mode='wb') as fdst:
        shutil.copyfileobj(stream, fdst)